In [2]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import ssl

def create_analysis_link(link):
    new = 'https://finance.naver.com' + link
    return 'https://finance.naver.com' + link[:6] + 'coinfo' + link[10:]

context = ssl._create_unverified_context()
table = []
for page in range(1,5):
    html = urlopen('https://finance.naver.com/sise/sise_market_sum.naver?&page='+str(page))
    bs = BeautifulSoup(html, 'html.parser')
    links = bs.findAll('a',{'href' : re.compile('^(/item/main.)')})


    for link in links:
            if 'href' in link.attrs:
                table.append(link.attrs['href'])
            
for i in range(len(table)):
    table[i] = str(create_analysis_link(table[i]))
    
print(table[180])

https://finance.naver.com/item/coinfo.naver?code=381970


In [3]:
# 얻어온 재무 분석 링크에서 javascript 링크 따오기.

array = []
for one in table:
    html = urlopen("{}".format(one), context = context)
    bs = BeautifulSoup(html,'html.parser')
    link = bs.find('div', {'class' : 'section inner_sub'}).find('iframe')
    if 'src' in link.attrs:
        goal = link.attrs['src']
        array.append(goal)
        
print(array[140])


https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd=004000


In [4]:
# 따온 링크에서 원하는 정보 추출하기

import csv

def make_name(arr):
    return arr[6][12:] + ' ' + 'ETF'

def extract_name(name):
    i = -1
    for i in range(1, len(name)):
        if name[-i] == '(':
            name = name[-i+1:-1]
            break
    return name

csvFile = open('COSPI_200.csv', 'wt+')
writer = csv.writer(csvFile)
try:

    writer.writerow(('순위', '종목 이름', 'EPS', 'BPS', 'PER', '업종PER', 'PBR', '현금배당수익률'))
    
    i=1
    for one in array:
        html = urlopen("{}".format(one))
        bs = BeautifulSoup(html, 'html.parser')
        try:
            table = bs.findAll('table',{'class' : 'cmp-table'})[0]
            rows = table.findAll('tr')[2].findAll('dt')

            name = bs.find('title').get_text()
            name = extract_name(name)

            data = []
            for row in rows:
                if row.get_text() == '12월 결산':
                    continue
                temp = row.get_text().split()
                if len(temp) == 1:
                    temp.append('N/A')
                data.append(temp[1])
        except IndexError as e:
            bs = bs.findAll('script', {'type':'text/javascript'})[2]
            arr = list(map(str,str(bs).split()))
            
            name = make_name(arr)
            data = ['N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A']
        
        finally:
            writer.writerow((i, name, data[0],data[1],data[2],data[3],data[4],data[5]))
            i += 1
    
finally:
    csvFile.close()
    
print('the end...')

the end...


In [ ]:
"""
나름 소기의 목적을 달성했다.

1위부터 200위 까지의 간단한 재무 정보를 모두 처리했다.
다만, etf나 레버리지 종목의 경우 재무 정보가 없기 때문에 모든 정보를 N/A 처리하고
이름만 뽑아내려고 했다.

그런데, 이름 뽑기가 생각보다 쉽지 않다. 
아무래도 html이 아니라 javascrpit로 작성되어 있기 때문이다.
하지만, 알고리즘만 잘다루더라도 쉽게 뽑아낼 수 있을 것 같은데, 우선 중요한 것은 아니니 얼버무리고 넘겼다.
구차한 알고리즘으로 풀고 싶지는 않다..
깔끔한 알고리즘으로 풀거나, 시간이 좀 걸리더라도 셀레니움으로 깔끔히 풀고 싶다.

또한 앞으로 '현명한 투자자'의 내용을 테스트하기 위해서는 남은 과제가 많다.

앞으로는 재무제표를 띄워서 순운전자본의 값도 구하고,
PER 값을 몇 년 평균으로 구하기도 하는 것이다. 
꾸준히 쌓아가자!
"""